In [ ]:

a_file="data/camera_event_A.csv"
from util import kafkaProducer
producer_A=kafkaProducer(a_file, "KAFKA_SERVER", "A", "cameraA", 5)
producer_A.produce_batches()

[INFO] Successfully loaded data from data/camera_event_A.csv
[INFO] Successfully connected to Kafka at 172.22.32.1
[INFO] Publishing batch #1 (20 records)...
[WARN] Failed to send event d40c586c-5be6-4743-a1e3-2269d9edaa72: KafkaTimeoutError: Failed to update metadata after 60.0 secs.
